In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [154]:
dataset='sms'

# from data.imdb_loader import DataLoader
# from data.youtube_loader import DataLoader
# from data.trec_loader import DataLoader
# from data.sms_loader import DataLoader
# from data.census_loader import DataLoader
dl = DataLoader()
_, _, _, train_ground, val_ground, test_ground, train_text, val_text, test_text = dl.load_data(dataset=dataset)
# _, _, _, train_ground, val_ground, test_ground, _,_,_,train_text, val_text, test_text= dl.load_data(dataset=dataset)
fname = './generated_data/'+dataset+'/normal_reef.npy'
train_reef = np.load(fname)

9762
common_idx 266


In [3]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')


In [5]:

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

I0331 19:23:04.758470 139706931414848 filelock.py:274] Lock 139706781793416 acquired on /home/ayusham/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

I0331 19:23:06.526269 139706931414848 filelock.py:318] Lock 139706781793416 released on /home/ayusham/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock
I0331 19:23:08.479745 139706931414848 filelock.py:274] Lock 139706781793416 acquired on /home/ayusham/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

I0331 19:23:09.142556 139706931414848 filelock.py:318] Lock 139706781793416 released on /home/ayusham/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock
I0331 19:23:09.787319 139706931414848 filelock.py:274] Lock 139706781793416 acquired on /home/ayusham/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


...

I0331 19:23:12.007874 139706931414848 filelock.py:318] Lock 139706781793416 released on /home/ayusham/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock
I0331 19:23:12.697202 139706931414848 filelock.py:274] Lock 139706781791288 acquired on /home/ayusham/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock


Downloading:   0%|          | 0.00/442 [00:00<?, ?B/s]

I0331 19:23:13.360728 139706931414848 filelock.py:318] Lock 139706781791288 released on /home/ayusham/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.d423bdf2f58dc8b77d5f5d18028d7ae4a72dcfd8f468e81fe979ada957a8c361.lock
I0331 19:23:14.026768 139706931414848 filelock.py:274] Lock 139694454647776 acquired on /home/ayusham/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

I0331 19:23:27.148020 139706931414848 filelock.py:318] Lock 139694454647776 released on /home/ayusham/.cache/huggingface/transformers/9c169103d7e5a73936dd2b627e42851bec0831212b677c637033ee4bce9ab5ee.126183e36667471617ae2f0835fab707baa54b731f991507ebbb55ea85adb12a.lock


In [155]:
df_train = pd.DataFrame(train_text)[0:83]

In [156]:
tokenized = df_train[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

train_features = last_hidden_states[0][:,0,:].numpy()

In [157]:
# test_text= np.delete(test_text , (138), axis=0) # For Youtube dataset

In [158]:
# test_ground = np.delete(test_ground , (138), axis=0) # For Youtube dataset

In [159]:
test_text.shape

(500,)

In [160]:
df_test = pd.DataFrame(test_text)
tokenized = df_test[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
k = 0
for i in tokenized.values:
    if len(i) > 500:
        print(len(i),k)
    k+=1
    if len(i) > max_len:
        max_len = len(i)
print('max len ', max_len)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

test_features = last_hidden_states[0][:,0,:].numpy()

max len  72


In [161]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_ground[0:83])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [162]:
lr_clf.score(test_features, test_ground)

0.918

In [165]:
test_ground.shape

(500,)

In [163]:
pred = lr_clf.predict(test_features)

In [164]:
from sklearn import metrics
print('Accuracy : ', np.sum(pred==test_ground)/len(test_ground))
print ('Test F1 Score: ', metrics.f1_score(test_ground, pred, average='binary'))
print ('Test Precision: ', metrics.precision_score(test_ground, pred, average='binary'))
print ('Test Recall: ', metrics.recall_score(test_ground, pred, average='macro'))

Accuracy :  0.918
Test F1 Score:  0.9164969450101833
Test Precision:  0.9574468085106383
Test Recall:  0.9189613217213115
